In [ ]:
import pandas as pd
import openpyxl

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
#read data from excel file
drdata_df = pd.read_excel('drdata.xlsx')
drq_df = pd.read_excel('drq.xlsx')
alldata_df = pd.read_excel('alldata.xlsx')

tv2data_df = pd.read_excel('tv2data.xlsx')
tv2q_df = pd.read_excel('tv2q.xlsx')

candidate_df = pd.read_excel('electeddata.xlsx')

In [ ]:
#count the number of unique values in all columns of drq_df
drq_df.nunique()

In [ ]:
#count the number of unique values in all columns of tv2q_df
print(tv2q_df.nunique())

# #print uniqe values in all columns of tv2q_df
# for col in tv2q_df.columns:
#     print(col, tv2q_df[col].unique())

#print header and count of question in tv2q_df where count is greater than 1
q = tv2q_df['question'].value_counts()
#print rows from tv2q_df where question is in q
tv2q_df[tv2q_df['question'].isin(q.index[q.gt(1)])]

In [ ]:
alldata_df.head()

In [ ]:
candidate_df.head()

In [ ]:
X = alldata_df

#parti handling
# X = pd.get_dummies(X, columns=['parti'])
# represent each parti with a number
X['parti_id'] = X['parti'].factorize()[0]

#drop non-numeric columns from alldata_df
for col in X.columns:
    if X[col].dtype == 'object' :
        X = X.drop(columns=[col])

#drop age column from X
X = X.drop(columns=['alder'])

In [ ]:
#questions that influence the result the most:
#correlation matrix for parti-id and all other columns sorted ascending
corr_matrix = X.corr()
print(corr_matrix['parti_id'].sort_values(ascending=False))

In [ ]:
#count the number of rows where age is 0
# candidate_df[candidate_df['alder'] == 0]
candidate_df.loc[candidate_df['alder'] == 0, 'alder'].count()

In [ ]:
#replace age 0 with median of the corresponding parti
candidate_df.loc[candidate_df['alder'] == 0, 'alder'] = candidate_df.groupby('parti')['alder'].transform('median')
candidate_df.loc[candidate_df['alder'] == 0, 'alder'].count()

In [ ]:
#average of all columns grouped by parti_id
candidate_df_without_name = candidate_df.drop(columns=['navn', 'storkreds'])
candidate_df_without_name.groupby('parti').mean()

In [ ]:
#average and median of age grouped by parti
candidate_df_without_name.groupby('parti').agg(['min','max','mean','median'])['alder']

In [ ]:
#plot the mean of the 555 column
candidate_df_without_name.groupby('parti').mean()['555'].plot(kind='bar')

In [ ]:
#plot the mean of the 543 column
candidate_df_without_name.groupby('parti').mean()['543'].plot(kind='bar')

In [ ]:
#group by parti and count number of occurences of -2 or 2 in all columns
candidate_df_without_name.groupby('parti').agg(lambda x: (x == -2).sum() + (x == 2).sum())

In [ ]:
#number of "strong" responses per candidate
candidates_frame = candidate_df.groupby('navn').agg(lambda x: (x == -2).sum() + (x == 2).sum()).sum(axis=1).to_frame('sum')
candidates_frame_with_party = candidates_frame.join(candidate_df.set_index('navn')['parti'])
candidates_frame_with_party.sort_values(by=['sum'], ascending=False)

In [ ]:
#boxplot of the 555 column grouped by parti
candidate_df_without_name.boxplot(column=['555'], by='parti', rot=90)

In [ ]:
#do PCA on the alldata_df and select the first 2 components
from sklearn import decomposition
pca = decomposition.PCA(n_components=None)
pca.fit(X)

#print first 5 explained variance ratios as float
for i in range(5):
    print(pca.explained_variance_ratio_[i])


In [ ]:
#plot a curve showing PCAs explained variance ratios
from matplotlib import pyplot as plt
import numpy as np

x = list(range(1, 1 + len(pca.explained_variance_ratio_)))
plt.plot(x, np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.show()

In [ ]:
#assign a numeric id to each parti
alldata_df['parti_id'] = pd.factorize(alldata_df['parti'])[0]

colors_palette = ['blue', 'green', 'orange', 'red', 'purple', 'pink', 'brown', 'gray', 'black', 'cyan', 'teal', 'pink', 'yellow', 'magenta', 'olive']
#assign a color to each parti based on the parti_id
alldata_df['color'] = alldata_df['parti_id'].apply(lambda x: colors_palette[x])

alldata_df[['parti', 'parti_id', 'color']].head()

In [ ]:
#plot the first 2 components of the PCA
pca2 = decomposition.PCA(n_components=2)
res2 = pca2.fit_transform(X)

#plot the first 2 components of the PCA with parti as color, add a legend with the parti names
# plt.scatter(res2[:, 0], res2[:, 1], c=alldata_df['color'])
for parti_id in alldata_df['parti_id'].unique():
    plt.scatter(res2[alldata_df['parti_id'] == parti_id, 0], res2[alldata_df['parti_id'] == parti_id, 1], c=colors_palette[parti_id])
#place a legend to the right of this smaller subplot
plt.legend(alldata_df['parti'].unique(), bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.show()

In [ ]:
res2_df = pd.DataFrame(res2, columns=['PC1', 'PC2'])
#add parti_id to res2_df
# res2_df['parti_id'] = alldata_df['parti_id']
#add parti to res2_df
res2_df['parti'] = alldata_df['parti']

#boxplot of the first 2 components of the PCA grouped by parti_id
res2_df.boxplot(column=['PC1'], by='parti', rot=90)

In [ ]:
res2_df.boxplot(column=['PC1'], by='parti', rot=90)